In [1]:
import numpy as np
import wandb
from torch.utils.data import Dataset, DataLoader
import os
import shutil
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as T
from tqdm.notebook import tqdm
from torchvision.models import densenet161, resnet50
import torch.nn as nn

In [2]:
os.environ["WANDB_API_KEY"] = '9c87219b80917d112c22c20a605d8dc6d6befdfd'

In [3]:

# https://github.com/isadrtdinov/intro-to-dl-hse/blob/2022-2023/seminars/202/seminar-04-fine-tuning.ipynb
class MyDataset(Dataset):
    
    def __init__(self, train=True, transform=T.ToTensor()):
        super().__init__()
        self.train = train
        self.transform = transform
        self.all_files = []
        self.images = []
        self.labels = {k:v[0] for k, v in pd.read_csv('../input/bhw-1-deep-learning/bhw1-dataset/labels.csv').set_index('Id').T.to_dict('list').items()}
        self.files = os.listdir('../input/bhw-1-deep-learning/bhw1-dataset/trainval/')
        TRAIN_SIZE = 0.8
        
        np.random.seed(812)
        np.random.shuffle(self.files)
    
        split_bound = int(len(self.files) * TRAIN_SIZE)
        
        train_files, test_files = self.files[:split_bound], self.files[split_bound:]

        if self.train:
            self.all_files += train_files
            train_labels = [self.labels[f_name] for f_name in train_files]

        else:
            self.all_files += test_files
            test_labels = [self.labels[f_name] for f_name in test_files]

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, item):
        filename = self.files[item]
        label = self.labels[filename]

        image = Image.open(os.path.join('../input/bhw-1-deep-learning/bhw1-dataset/trainval/', filename)).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        return image, label
    

In [4]:
# https://github.com/isadrtdinov/intro-to-dl-hse/blob/2022-2023/seminars/202/seminar-04-fine-tuning.ipynb
def training_epoch(model, optimizer, criterion, train_loader, tqdm_desc, device):
    train_loss, train_accuracy = 0.0, 0.0

    model.train()

    for images, labels in tqdm(train_loader, desc=tqdm_desc):
        images = images.to(device)  
        labels = labels.to(device) 
        
        optimizer.zero_grad()
        logits = model(images) 
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.shape[0]
        train_accuracy += (logits.argmax(dim=1) == labels).sum().item()
        #print(train_loss)
    
    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    return train_loss, train_accuracy


@torch.no_grad()
def validation_epoch(model, criterion, test_loader, tqdm_desc, device):
    test_loss, test_accuracy = 0.0, 0.0
    model.eval()
    for images, labels in tqdm(test_loader, desc=tqdm_desc):
        images = images.to(device)  
        labels = labels.to(device)  
        logits = model(images)
        loss = criterion(logits, labels)

        test_loss += loss.item() * images.shape[0]
        test_accuracy += (logits.argmax(dim=1) == labels).sum().item()


    test_loss /= len(test_loader.dataset)
    test_accuracy /= len(test_loader.dataset)

    return test_loss, test_accuracy

    
def train(model, optimizer, scheduler, criterion, train_loader, test_loader, num_epochs, device):
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []
    wandb.init(project="BHW-1-DL", entity="tsessk")
    prev_acc = -1
    
    for epoch in range(1, num_epochs + 1):

        train_loss, train_accuracy = training_epoch(
            model, optimizer, criterion, train_loader,
            tqdm_desc=f'Training {epoch}/{num_epochs}',
            device=device
        )
        
        test_loss, test_accuracy = validation_epoch(
            model, criterion, test_loader,
            tqdm_desc=f'Validating {epoch}/{num_epochs}',
            device=device
        )

        if scheduler is not None:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(test_loss)
            else:
                scheduler.step()

        train_losses += [train_loss]
        train_accuracies += [train_accuracy]
        test_losses += [test_loss]
        test_accuracies += [test_accuracy]
        
        if test_accuracy > prev_acc:
            torch.save(model, f'model_acc_{test_accuracy}.pt')
            print('better acc =', test_accuracy)
            prev_acc = test_accuracy
        
        print(f'{epoch}: train_acc = {train_accuracy}, train_loss = {train_loss}, test_accuracy = {test_accuracy}, test_loss = {test_loss}')
        wandb.log({'train_accuracy': train_accuracy,
                   'train_loss': train_loss,
                   'test_accuracy': test_accuracy,
                   'test_loss': test_loss})
        
    return train_losses, test_losses, train_accuracies, test_accuracies

In [5]:
class MyDenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = resnet50(pretrained=False).to(device)
        self.model.fc = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.Linear(in_features=1000, out_features=200, bias=True)
        )

    def forward(self, x):
        return self.model(x)

    def predict(self, x):
        logits = self.forward(x)
        return logits.argmax(dim=1)

In [6]:
train_transform = T.Compose([
        T.RandomResizedCrop(224),
        T.RandomHorizontalFlip(),
        T.ColorJitter(0.5, 0.5, 0.5, 0.5),
        T.RandomApply([
            T.RandomAffine(degrees=10)], p=0.33),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.RandomErasing(p=0.1)
    ])

val_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [7]:
train_dataset, test_dataset = MyDataset(train=True, transform=train_transform), MyDataset(train=False, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 70
model = MyDenseNet().to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=4, min_lr=0, verbose=True)
#torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, np.floor(len(train_loader) / 64))



wandb_r = wandb.init(project='BHW-1-DL', entity='tsessk')

train_losses, test_losses, train_accuracies, test_accuracies = train(
    model,
    optimizer,
    scheduler,
    criterion,
    train_loader,
    test_loader,
    num_epochs,
    device
)
wandb_r.finish()

wandb: Currently logged in as: tsessk. Use `wandb login --relogin` to force relogin


Training 1/70:   0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#!g1.1
class MyDataset_test(Dataset):

    def __init__(self, transform=T.ToTensor()):
        super().__init__()
        self.transform = transform
        self.files = os.listdir('../input/bhw-1-deep-learning/bhw1-dataset/test/')
    

    def __len__(self):
        return len(self.files)

    def __getitem__(self, item):
        filename = self.files[item]

        image = Image.open(os.path.join('../input/bhw-1-deep-learning/bhw1-dataset/test/', filename)).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        return image, filename

In [ ]:
test_dataset_ = MyDataset_test(transform=val_transform)

In [ ]:
test_loader_ = DataLoader(test_dataset_, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


dict_test_pred = dict()

model.eval()
with torch.no_grad():
    for images, filenames in tqdm(test_loader_, desc=None):
        images = images.to(device)  
        logits = model(images)

        preds = logits.argmax(dim=1)
        dict_test_pred.update({file_:preds[i].item() for i, file_ in enumerate(filenames)})


In [ ]:
from collections import OrderedDict


dict_test_pred = OrderedDict(sorted(dict_test_pred.items()))

In [ ]:
labels_test = pd.DataFrame.from_dict({'Id' : dict_test_pred.keys(), 'Label' : dict_test_pred.values()})

In [ ]:
labels_test.to_csv('labels_test_4.csv', index=False)